# Species distribution modeling with multimodal satellite and environmental data

Welcome to this wonderfull main to discover the beaty of mother earth, enjoy your work and be happy :)

## 1. Setup

## 2. Data loading

## 3. Create model

## 4. Evaluation of the model